In [3]:
import csv
import os
import re
import tensorflow as tf

from object_detection import evaluator
from object_detection.core import standard_fields
from object_detection.metrics import tf_example_parser
from object_detection.utils import config_util
from object_detection.utils import label_map_util

import numpy as np
# flags = tf.app.flags
# tf.logging.set_verbosity(tf.logging.INFO)

# flags.DEFINE_string('eval_dir', None, 'Directory to write eval summaries to.')
# flags.DEFINE_string('eval_config_path', None,
#                     'Path to an eval_pb2.EvalConfig config file.')
# flags.DEFINE_string('input_config_path', None,
#                     'Path to an eval_pb2.InputConfig config file.')

# FLAGS = flags.FLAGS
import cv2
from utils import visualization_utils as vis_util
from matplotlib import pyplot as plt
%matplotlib inline




In [16]:
PATH_TO_LABELS = os.path.join('/root/models/research/object_detection/data', 'oid_bbox_trainable_label_map_add2.pbtxt')
NUM_CLASSES = 547
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index547 = label_map_util.create_category_index(categories) # int: id, name
category_dict547 = label_map_util.get_label_map_dict(PATH_TO_LABELS) # /m* to int



In [40]:
PATH_TO_LABELS = os.path.join('/root/models/research/object_detection/data', 'oid_v4_label_map.pbtxt')
NUM_CLASSES = 601
label_map600 = label_map_util.load_labelmap(PATH_TO_LABELS)
categories600 = label_map_util.convert_label_map_to_categories(label_map600, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index600 = label_map_util.create_category_index(categories600) # int: id, name
category_dict600 = label_map_util.get_label_map_dict(PATH_TO_LABELS) # /m* to int




In [41]:
## bbox_labels_600_hierarchy.json
import json

with open('/root/bbox_labels_600_hierarchy.json') as json_file:
    data = json.load(json_file)

In [42]:
data.keys()

[u'Subcategory', u'LabelName']

In [43]:
category_index600[category_dict600[data['Subcategory'][0]['LabelName']]]

{'id': 132, 'name': u'Coin'}

In [44]:
data['Subcategory'][0]['LabelName']

u'/m/0242l'

In [48]:
isinstance(data['Subcategory'], list)

True

In [54]:
def dfs(node, result):
    if isinstance(node, list):
        for subDict in node:
            dfs(subDict, result)
    elif isinstance(node, dict):
        if 'Subcategory' in node.keys():
            dfs(node['Subcategory'], result)
        else:
            result.append(node['LabelName'])


In [61]:
child_labels = []
dfs(data, child_labels)

In [62]:
len(child_labels)

590

In [63]:
len(set(child_labels))

525

In [64]:
child_labels = list(set(child_labels))

In [68]:
parents_IDs = set()
for key, label_int in category_dict547.items():
    if key not in child_labels:
        print(category_index547[label_int])
        parents_IDs.add(category_index547[label_int]['id'])

{'id': 18, 'name': u'Furniture'}
{'id': 53, 'name': u'Insect'}
{'id': 302, 'name': u'Bathroom accessory'}
{'id': 2, 'name': u'Clothing'}
{'id': 140, 'name': u'Bed'}
{'id': 28, 'name': u'Sports equipment'}
{'id': 8, 'name': u'Vehicle'}
{'id': 89, 'name': u'Ball'}
{'id': 155, 'name': u'Home appliance'}
{'id': 512, 'name': u'Personal care'}
{'id': 15, 'name': u'Food'}
{'id': 246, 'name': u'Cosmetics'}
{'id': 61, 'name': u'Baked goods'}
{'id': 117, 'name': u'Reptile'}
{'id': 156, 'name': u'Marine invertebrates'}
{'id': 171, 'name': u'Plumbing fixture'}
{'id': 13, 'name': u'Animal'}
{'id': 10, 'name': u'Land vehicle'}
{'id': 41, 'name': u'Watercraft'}
{'id': 142, 'name': u'Seafood'}
{'id': 83, 'name': u'Fast food'}
{'id': 26, 'name': u'Auto part'}
{'id': 239, 'name': u'Kitchen utensil'}
{'id': 233, 'name': u'Bear'}
{'id': 6, 'name': u'Plant'}
{'id': 14, 'name': u'Car'}
{'id': 9, 'name': u'Building'}
{'id': 22, 'name': u'Fashion accessory'}
{'id': 151, 'name': u'Marine mammal'}
{'id': 178, '

In [70]:
list(parents_IDs)

[512,
 1,
 2,
 5,
 6,
 263,
 8,
 9,
 10,
 23,
 140,
 13,
 142,
 15,
 144,
 17,
 18,
 22,
 151,
 281,
 26,
 155,
 156,
 158,
 31,
 35,
 34,
 27,
 36,
 166,
 49,
 41,
 171,
 172,
 50,
 302,
 48,
 177,
 178,
 53,
 56,
 57,
 159,
 61,
 63,
 65,
 11,
 292,
 72,
 73,
 203,
 45,
 28,
 79,
 80,
 83,
 139,
 14,
 216,
 89,
 71,
 219,
 97,
 187,
 100,
 103,
 104,
 233,
 272,
 239,
 117,
 246,
 189,
 252]

# create label map
oid_bbox_547_removeParent.pbtxt

In [79]:
child_dict = {}
for key, label_int in category_dict547.items():
    if key in child_labels:
        child_dict[label_int]={'id':label_int, 'name':key, 'display_name':category_index547[label_int]['name']}
#         print(category_index547[label_int])
#         parents_IDs.add(category_index547[label_int]['id'])

In [80]:
child_dict_sort = {}
cnt=1
for key , item in child_dict.items():
    item['id']=cnt
    child_dict_sort[cnt]=item
    cnt+=1

In [81]:
child_dict_sort

{1: {'display_name': u'Man', 'id': 1, 'name': u'/m/04yx4'},
 2: {'display_name': u'Face', 'id': 2, 'name': u'/m/0dzct'},
 3: {'display_name': u'Woman', 'id': 3, 'name': u'/m/03bt1vf'},
 4: {'display_name': u'Girl', 'id': 4, 'name': u'/m/05r655'},
 5: {'display_name': u'Wheel', 'id': 5, 'name': u'/m/083wq'},
 6: {'display_name': u'Window', 'id': 6, 'name': u'/m/0d4v4'},
 7: {'display_name': u'House', 'id': 7, 'name': u'/m/03jm5'},
 8: {'display_name': u'Boy', 'id': 8, 'name': u'/m/01bl7v'},
 9: {'display_name': u'Glasses', 'id': 9, 'name': u'/m/0jyfg'},
 10: {'display_name': u'Suit', 'id': 10, 'name': u'/m/01xyhv'},
 11: {'display_name': u'Dress', 'id': 11, 'name': u'/m/01d40f'},
 12: {'display_name': u'Dog', 'id': 12, 'name': u'/m/0bt9lr'},
 13: {'display_name': u'Human body', 'id': 13, 'name': u'/m/02p0tk3'},
 14: {'display_name': u'Jeans', 'id': 14, 'name': u'/m/0fly7'},
 15: {'display_name': u'Hair', 'id': 15, 'name': u'/m/03q69'},
 16: {'display_name': u'Tire', 'id': 16, 'name': u'

# save to pbtxt